# Prepare working dataset for search and vis

In [1]:
import json, os
import numpy as np
import pandas as pd
import nltk

## Load json file with parsed information

In [2]:
# paths
path_to_json_file = os.path.abspath('../data/json/')

### json file
# json_file = '/all_parsed_data_json' #name json file  
json_file = '/sample_json' #sample json file for testing  

file = '{}{}.json'.format(path_to_json_file, json_file)

In [3]:
# load json file
df = pd.read_json(file)

In [4]:
# Display full dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
df.head()

,nct_id,study_first_submitted,source,brief_title,overall_status,verification_date,study_type,study_first_posted,last_update_submitted,last_update_posted,phase,condition,condition_browse/mesh_term,intervention_browse/mesh_term,detailed_description/textblock,brief_summary/textblock,location/facility/address/city,location/facility/address/country,location/facility/address/zip,sponsors/lead_sponsor/agency,sponsors/lead_sponsor/agency_class,study_design_info/allocation,study_design_info/intervention_model,study_design_info/primary_purpose
0,NCT00058058,"April 7, 2003",American College of Radiology Imaging Network,Magnetic Resonance Imaging in Women Recently D...,Completed,June 2018,Interventional,"April 9, 2003","June 29, 2018","July 27, 2018",N/A,Breast Cancer,Breast Neoplasms,None,\n OBJECTIVES:\n\n - Determine th...,\n RATIONALE: Diagnostic procedures such ...,Little Rock,United States,72205,American College of Radiology Imaging Network,Other,None,Single Group Assignment,Diagnostic
1,NCT01980602,"September 24, 2013",University of Hull,Effect of Exercise on Patients With Claudicati...,Unknown status,November 2013,Interventional,"November 11, 2013","November 4, 2013","November 11, 2013",N/A,Intermittent Claudication,Peripheral Arterial Disease,None,None,\n Title: How does exercise improve the c...,Hull,United Kingdom,HU3 2JZ,University of Hull,Other,Randomized,Single Group Assignment,Treatment
10,NCT01277458,"January 14, 2011",Public Health England,Ethnicity Data in HIV Positive Men Who Have Se...,Unknown status,December 2010,Observational,"January 17, 2011","January 14, 2011","January 17, 2011",Phase 4,HIV Infections,HIV Infections,None,\n HIV Doctors will be identified at each...,\n Are there differences between the way ...,London,United Kingdom,SE1 7EH,Public Health England,Other,None,None,None
100,NCT01164891,"July 16, 2010",Hoffmann-La Roche,A Pharmacokinetic and Metabolism Study of 14C-...,Completed,September 2017,Interventional,"July 19, 2010","January 11, 2016","February 5, 2016",Phase 3,Malignant Melanoma,Melanoma,None,None,"\n This open-label, non-randomized study ...",Zürich,Switzerland,8091,Hoffmann-La Roche,Industry,None,Single Group Assignment,Treatment
1000,NCT03795727,"January 3, 2019",Cairo University,Clinical Evaluation of Sectional Matrix Versus...,Not yet recruiting,February 2019,Interventional,"January 8, 2019","March 10, 2019","March 12, 2019",N/A,Proximal Contact,None,None,None,\n The aim of this study is to evaluate t...,Cairo,Egypt,11553,Cairo University,Other,Randomized,Parallel Assignment,Treatment


## Clean dataset

In [6]:
# change column type
def data_types(dataframe, cols = [], to_type = ''):
    for col in cols:
        dataframe[col] = df[col].astype(to_type)

In [7]:
columns_dates = ['study_first_submitted', 'last_update_submitted', 'verification_date']
data_types(df, columns_dates, 'datetime64')

In [8]:
# Extract year 
df['year_submitted'] = df['study_first_submitted'].dt.year
df['year_last_updated'] = df['last_update_submitted'].dt.year
df['year_verification'] = df['verification_date'].dt.year

In [9]:
# Remove unnecesary columns
df.drop(columns_dates, axis = 1, inplace=True)

In [10]:
columns_to_drop = ['study_first_posted', 'last_update_posted']
df.drop(columns_to_drop, axis = 1, inplace=True)

In [11]:
# Remove /n in all df
df = df.replace(r'\n',' ', regex=True)

In [12]:
# Create new column for all text
df['all_text'] = df['source'] + ' ' + df['brief_title'] + ' ' + df['condition'] + ' ' + df['condition_browse/mesh_term'] + ' '+ df['intervention_browse/mesh_term'] + ' '+ df['detailed_description/textblock'] + ' ' + df['brief_summary/textblock']

# All_text in lowercase
df['all_text'] = df['all_text'].str.lower()

# remove extra whitespace
df.all_text = df.all_text.replace('\s+', ' ', regex=True)


In [13]:
# Remove extra white space in summary
df['brief_summary/textblock'] = df['brief_summary/textblock'].replace('\s+', ' ', regex=True)
df['brief_summary/textblock'][0]

' RATIONALE: Diagnostic procedures such as magnetic resonance imaging (MRI) may improve the ability to detect cancer in the unaffected breast of women recently diagnosed with unilateral breast cancer. PURPOSE: Diagnostic trial to determine the effectiveness of MRI in evaluating the unaffected breast of women recently diagnosed with unilateral breast cancer. '

In [14]:
# # add url columns
url_string = 'https://clinicaltrials.gov/ct2/show/'
df['col'] = url_string + df['nct_id'].astype(str)

In [15]:
df.sample(3)

,nct_id,source,brief_title,overall_status,study_type,phase,condition,condition_browse/mesh_term,intervention_browse/mesh_term,detailed_description/textblock,brief_summary/textblock,location/facility/address/city,location/facility/address/country,location/facility/address/zip,sponsors/lead_sponsor/agency,sponsors/lead_sponsor/agency_class,study_design_info/allocation,study_design_info/intervention_model,study_design_info/primary_purpose,year_submitted,year_last_updated,year_verification,all_text,col
7171,NCT03100630,Hoffmann-La Roche,Compare Bioavailability of RO7239361 After Sub...,Completed,Interventional,None,Healthy,None,None,None,Randomized study in healthy men and women. As...,Cypress,United States,90630,Hoffmann-La Roche,Industry,Randomized,Parallel Assignment,Other,2017,2018,2016.0,hoffmann-la roche compare bioavailability of r...,https://clinicaltrials.gov/ct2/show/NCT03100630
6032,NCT01619553,UConn Health,Genetic Analysis of Keloids,Recruiting,Observational,N/A,Keloid,Keloid,None,Keloids are scars that keep growing bey...,Keloids have a strong genetic component. The ...,Farmington,United States,06030-3705,UConn Health,Other,None,None,None,2012,2018,2016.0,uconn health genetic analysis of keloids keloi...,https://clinicaltrials.gov/ct2/show/NCT01619553
4651,NCT03649958,"Brain State Technologies, LLC",Neurotechnology Following Traumatic Brain Injury,Recruiting,Interventional,Phase 4,Post-Concussion Symptoms,Brain Injuries,None,Persistent symptoms after concussion or...,This study seeks to improve symptoms such as ...,Bethesda,United States,20814,"Brain State Technologies, LLC",Industry,Randomized,Parallel Assignment,Treatment,2018,2018,2010.0,"brain state technologies, llc neurotechnology ...",https://clinicaltrials.gov/ct2/show/NCT03649958


## Preprocess dataset
- Tokenize, Lemmatize / Stem
- Remove stopwords

In [16]:
from nltk.stem import PorterStemmer
from nltk import WordNetLemmatizer

# porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

In [17]:
# Stemming
# def stem_sentences(text):
#     tokens = text.split()
#     stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
#     return ' '.join(stemmed_tokens)

# df['stems'] = df['all_text'].apply(stem_sentences)

In [18]:
# Lemmatizing
def lemm_sentences(text):
    tokens = text.split()
    lemm_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemm_tokens)

df['lemmas'] = df['all_text'].apply(lemm_sentences)

In [19]:
# Remove stop words [not needed with countvectorizer]
# stop_words = stopwords.words('english')
# df['tokens'] = df['lemmas'].apply(lambda x: [item for item in x.split() if item not in stop_words])

In [20]:
df.lemmas[0:5]

0       american college of radiology imaging network ...
1       university of hull effect of exercise on patie...
10      public health england ethnicity data in hiv po...
100     hoffmann-la roche a pharmacokinetic and metabo...
1000    cairo university clinical evaluation of sectio...
Name: lemmas, dtype: object

## Bags of words

In [71]:
# Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re

In [79]:
pat_numbers = lambda x: re.sub(r'(\d)+', '', x.lower())

cv = CountVectorizer(stop_words='english',
                     preprocessor = pat_numbers,
                     max_features = 1800,
                     lowercase = True,
                     max_df = 0.8,
                     ngram_range = (1, 3))

In [80]:
docs = list(df['lemmas'])

X = cv.fit_transform(docs)

In [81]:
len(cv.vocabulary_)

1800

In [82]:
word_counts = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

In [83]:
# add nct_id to word_counts df
nct_id = df['nct_id']

df_word_counts = pd.merge(nct_id, word_counts, left_index=True, right_index=True)

In [84]:
df_word_counts.head()

,nct_id,abdominal,ability,ablation,able,abnormal,abnormality,absence,absence disease,absence disease progression,acceptability,access,according,account,accrual,accuracy,achieve,achieved,acid,act,action,activation,active,activity,acupuncture,acute,ad,added,addition,additional,additionally,address,adenocarcinoma,adequate,adherence,adjuvant,administered,administration,admission,admitted,adolescent,adult,adults,advanced,advantage,adverse,adverse event,adverse events,affect,affected,african,age,aged,agent,aid,aim,aim study,aimed,airway,al,alcohol,algorithm,allergic,allergy,allocated,allogeneic,allow,allowed,allows,alpha,alternative,alzheimer,alzheimer disease,american,analgesia,analgesic,analysis,analyzed,anemia,anesthesia,anesthetic,animal,anterior,anti,anti inflammatory,antibiotic,antibody,antigen,antiretroviral,anxiety,aortic,apnea,app,application,applied,approach,appropriate,approved,approximately,area,arm,arms,art,arterial,artery,artery disease,arthritis,arthroplasty,asked,aspect,aspirin,ass,ass effect,ass efficacy,ass safety,assay,assessed,assessing,assessment,assigned,assisted,associated,association,asthma,atrial,atrial fibrillation,attempt,attention,autologous,available,average,avoid,background,bacteria,bacterial,balance,barrier,based,baseline,basic,basis,begin,beginning,behavior,behavioral,beneficial,benefit,best,beta,better,bevacizumab,biological,biomarkers,biopsy,birth,bladder,bleeding,blind,blind placebo,blind placebo controlled,blinded,block,blood,blood flow,blood glucose,blood pressure,blood sample,bmi,body,body weight,bone,bone marrow,bortezomib,bowel,bp,brain,breast,breast cancer,breast cancer breast,breast neoplasm,breathing,brief,bupivacaine,burden,bypass,calcium,calculated,calculation,called,cancer,cancer breast,cancer cell,cancer center,cancer patient,candidate,capacity,capecitabine,capsule,carboplatin,carcinoma,cardiac,cardiac surgery,cardiovascular,cardiovascular disease,care,care unit,caregiver,carried,case,cases,catheter,cause,caused,cbt,cc,cd,cell,cell carcinoma,cell lung,cell lung cancer,cell transplantation,cells,cellular,center,central,centre,cerebral,certain,cervical,cessation,challenge,change,characteristic,characterize,characterized,check,chemotherapy,chest,child,childhood,children,china,chinese,choice,cholesterol,chronic,chronic obstructive,chronic obstructive pulmonary,circulating,cisplatin,...,statistical analysis,status,stay,stem,stem cell,stem cell transplantation,stenosis,stent,step,steroid,stiffness,stimulation,stop,stored,strategy,stratified,strength,stress,stroke,structure,student,studied,studies,study aim,study ass,study compare,study conducted,study design,study designed,study determine,study drug,study evaluate,study evaluate efficacy,study evaluate safety,study examine,study ha,study investigate,study investigator,study participant,study patient,study population,study shown,study study,study subject,study test,study treatment,study visit,study wa,studying,sub,subcutaneous,subject,subject receive,subjective,subjects,subsequent,substance,success,successful,suffering,sufficient,suggest,suggested,suggests,suicide,superior,supplement,supplementation,support,surface,surgeon,surgery,surgical,survey,survival,symptom,symptomatic,symptoms,syndrome,systemic,tablet,taken,taking,target,targeted,task,team,technique,technology,teeth,telephone,temozolomide,term,test,test hypothesis,tested,testing,testosterone,tests,th,therapeutic,therapy,therapy patient,thickness,thoracic,thought,threshold,thrombosis,thyroid,time,time point,tissue,tolerability,tolerance,tolerated,tolerated dose,tomography,tool,tooth,topical,total,total patient,toxicity,tract,traditional,trained,training,transfusion,transplant,transplantation,trauma,traumatic,treat,treated,treating,treating patient,treatment,treatment group,treatment patient,treatment period,treatments,trial,trial study,trials,tube,tumor,tumor cell,tumor cells,tumors,tumour,twice,twice daily,type,type diabetes,type diabetes mellitus,ulcer,ultrasound,unacceptable,unaccepta

_______

## Export working dataset

In [85]:
# Check all dataframes in space
%who DataFrame

df	 df_results	 df_word_counts	 word_counts	 


In [86]:
# Export datasets: df, df_word_counts (key: ntc_id)

path_to_working_datasets = os.path.abspath('../data/working_data')

try: 
    os.mkdir(path_to_working_datasets)
except:
    pass
    print('Error')

Error


### Upload data to sqlite db

In [41]:
import sqlite3

conn = sqlite3.connect('../data/working_data/database.db')

conn

In [65]:
# upload df to database
df.to_sql('all_trials', con=conn)

In [61]:
# test db
df_results = pd.read_sql_query("SELECT * from all_trials;", conn)
df_results.head()

,index,nct_id,source,brief_title,overall_status,study_type,phase,condition,condition_browse/mesh_term,intervention_browse/mesh_term,detailed_description/textblock,brief_summary/textblock,location/facility/address/city,location/facility/address/country,location/facility/address/zip,sponsors/lead_sponsor/agency,sponsors/lead_sponsor/agency_class,study_design_info/allocation,study_design_info/intervention_model,study_design_info/primary_purpose,year_submitted,year_last_updated,year_verification,all_text,col,lemmas
0,0,NCT00058058,American College of Radiology Imaging Network,Magnetic Resonance Imaging in Women Recently D...,Completed,Interventional,N/A,Breast Cancer,Breast Neoplasms,None,OBJECTIVES: - Determine the d...,RATIONALE: Diagnostic procedures such as magn...,Little Rock,United States,72205,American College of Radiology Imaging Network,Other,None,Single Group Assignment,Diagnostic,2003,2018,2018.0,american college of radiology imaging network ...,https://clinicaltrials.gov/ct2/show/NCT00058058,american college of radiology imaging network ...
1,1,NCT01980602,University of Hull,Effect of Exercise on Patients With Claudicati...,Unknown status,Interventional,N/A,Intermittent Claudication,Peripheral Arterial Disease,None,None,Title: How does exercise improve the calf mus...,Hull,United Kingdom,HU3 2JZ,University of Hull,Other,Randomized,Single Group Assignment,Treatment,2013,2013,2013.0,university of hull effect of exercise on patie...,https://clinicaltrials.gov/ct2/show/NCT01980602,university of hull effect of exercise on patie...
2,10,NCT01277458,Public Health England,Ethnicity Data in HIV Positive Men Who Have Se...,Unknown status,Observational,Phase 4,HIV Infections,HIV Infections,None,HIV Doctors will be identified at each ...,Are there differences between the way that no...,London,United Kingdom,SE1 7EH,Public Health England,Other,None,None,None,2011,2011,2010.0,public health england ethnicity data in hiv po...,https://clinicaltrials.gov/ct2/show/NCT01277458,public health england ethnicity data in hiv po...
3,100,NCT01164891,Hoffmann-La Roche,A Pharmacokinetic and Metabolism Study of 14C-...,Completed,Interventional,Phase 3,Malignant Melanoma,Melanoma,None,None,"This open-label, non-randomized study will as...",Zürich,Switzerland,8091,Hoffmann-La Roche,Industry,None,Single Group Assignment,Treatment,2010,2016,2017.0,hoffmann-la roche a pharmacokinetic and metabo...,https://clinicaltrials.gov/ct2/show/NCT01164891,hoffmann-la roche a pharmacokinetic and metabo...
4,1000,NCT03795727,Cairo University,Clinical Evaluation of Sectional Matrix Versus...,Not yet recruiting,Interventional,N/A,Proximal Contact,None,None,None,The aim of this study is to evaluate the effe...,Cairo,Egypt,11553,Cairo University,Other,Randomized,Parallel Assignment,Treatment,2019,2019,2019.0,cairo university clinical evaluation of sectio...,https://clinicaltrials.gov/ct2/show/NCT03795727,cairo university clinical evaluation of sectio...


In [136]:
# Test string query
search_query = pd.read_sql_query("SELECT * from all_trials WHERE all_text LIKE '%breast cancer%';", conn)
len(search_query)

255

In [87]:
# upload count_vectorizer to database
# sqlite limit = 2000
df_word_counts.to_sql('word_counts', con=conn)

/Users/cms/Sites/env/haden/lib/python3.7/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [121]:
df_counts = pd.read_sql_query("SELECT * from word_counts;", conn)
df_counts.head()

,level_0,nct_id,abdominal,ability,ablation,able,abnormal,abnormality,absence,absence disease,absence disease progression,acceptability,access,according,account,accrual,accuracy,achieve,achieved,acid,act,action,activation,active,activity,acupuncture,acute,ad,added,addition,additional,additionally,address,adenocarcinoma,adequate,adherence,adjuvant,administered,administration,admission,admitted,adolescent,adult,adults,advanced,advantage,adverse,adverse event,adverse events,affect,affected,african,age,aged,agent,aid,aim,aim study,aimed,airway,al,alcohol,algorithm,allergic,allergy,allocated,allogeneic,allow,allowed,allows,alpha,alternative,alzheimer,alzheimer disease,american,analgesia,analgesic,analysis,analyzed,anemia,anesthesia,anesthetic,animal,anterior,anti,anti inflammatory,antibiotic,antibody,antigen,antiretroviral,anxiety,aortic,apnea,app,application,applied,approach,appropriate,approved,approximately,area,arm,arms,art,arterial,artery,artery disease,arthritis,arthroplasty,asked,aspect,aspirin,ass,ass effect,ass efficacy,ass safety,assay,assessed,assessing,assessment,assigned,assisted,associated,association,asthma,atrial,atrial fibrillation,attempt,attention,autologous,available,average,avoid,background,bacteria,bacterial,balance,barrier,based,baseline,basic,basis,begin,beginning,behavior,behavioral,beneficial,benefit,best,beta,better,bevacizumab,biological,biomarkers,biopsy,birth,bladder,bleeding,blind,blind placebo,blind placebo controlled,blinded,block,blood,blood flow,blood glucose,blood pressure,blood sample,bmi,body,body weight,bone,bone marrow,bortezomib,bowel,bp,brain,breast,breast cancer,breast cancer breast,breast neoplasm,breathing,brief,bupivacaine,burden,bypass,calcium,calculated,calculation,called,cancer,cancer breast,cancer cell,cancer center,cancer patient,candidate,capacity,capecitabine,capsule,carboplatin,carcinoma,cardiac,cardiac surgery,cardiovascular,cardiovascular disease,care,care unit,caregiver,carried,case,cases,catheter,cause,caused,cbt,cc,cd,cell,cell carcinoma,cell lung,cell lung cancer,cell transplantation,cells,cellular,center,central,centre,cerebral,certain,cervical,cessation,challenge,change,characteristic,characterize,characterized,check,chemotherapy,chest,child,childhood,children,china,chinese,choice,cholesterol,chronic,chronic obstructive,chronic obstructive pulmonary,circulating,cisplatin,class,clear,clinic,clinical,clinical outcome,clinical practice,clinical study,clinical trial,clinically,clinician,closure,cluster,cm,cns,cognition,cognitive,cohort,cohort study,collect,collected,collection,college,colon,colonoscopy,colorectal,colorectal cancer,combination,combined,come,committee,common,commonly,communication,community,comparative,compare,compared,compared placebo,comparing,comparison,complete,completed,completion,completion study,complex,compliance,complication,complications,component,composition,comprehensive,computer,concentration,concept,concern,condition,conditioning,conditions,conduct,conducted,confirm,confirmed,consecutive,consent,consequence,considered,consist,consisting,consists,consumption,contact,containing,content,context,continue,continued,continuous,contrast,contribute,control,control group,controlled,controlled study,controlled trial,controls,conventional,copd,cord,core,coronary,coronary artery,coronary artery disease,correlate,correlation,cost,counseling,count,country,course,cpap,cr,criteria,criterion,critical,cross,crossover,csf,ct,ct scan,culture,current,currently,cycle,cyclophosphamide,cytokine,daily,damage,data,data collected,data collection,database,date,day,day course,day day,day treatment,day week,days,death,decision,decline,decrease,decreased,deep,deficiency,deficit,define,defined,degree,delay,delayed,delirium,delivered,delivery,dementia,demographic,demonstrate,demonstrated,density,dental,department,dependent,depending,depression,depressive,depth,derived,described,design,designed,despite,detect,detection,determination,determine,determine effect,determine safety,dete

In [120]:
# List databases
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,all_trials
1,word_counts


In [137]:
conn.close()

--------

## TF-IDF

In [91]:
from sklearn.feature_extraction.text import TfidfTransformer

In [92]:
transformer = TfidfTransformer()
tweights = transformer.fit_transform(X)
tweights

<7941x1800 sparse matrix of type '<class 'numpy.float64'>'
	with 565159 stored elements in Compressed Sparse Row format>

In [93]:
# turn weights data into a dataframe
tf = pd.DataFrame(tweights.toarray(), columns=cv.get_feature_names())

In [94]:
# Top terms by average tf-idf weight
weights = np.asarray(tweights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cv.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(10)

,term,weight
1125,patient,0.054052
1686,treatment,0.036370
188,cancer,0.025435
443,disease,0.023751
1441,safety,0.022737
1601,subject,0.022428
462,dose,0.020823
649,group,0.020447
488,effect,0.019788
494,efficacy,0.019406


In [95]:
# Create new dataframe with nct-id and merge wth tf by index
pd.set_option('display.max_columns', None) 

df_tf = df['nct_id']
df_tf.head()

0       NCT00058058
1       NCT01980602
10      NCT01277458
100     NCT01164891
1000    NCT03795727
Name: nct_id, dtype: object

In [96]:
df_tf = pd.merge(df_tf, tf, left_index=True, right_index=True)

In [97]:
df_tf.head()

nct_id  abdominal   ability  ablation  able  abnormal  abnormality  absence  absence disease  absence disease progression  acceptability   access  according  account   accrual  accuracy  achieve  achieved  acid  act  action  activation  active  activity  acupuncture  acute   ad  added  addition  additional  additionally  address  adenocarcinoma  adequate  adherence  adjuvant  administered  administration  admission  admitted  adolescent  adult  adults  advanced  advantage  adverse  adverse event  adverse events  affect  affected  african       age  aged  agent  aid       aim  aim study  aimed  airway   al  alcohol  algorithm  allergic  allergy  allocated  allogeneic  allow  allowed  allows  alpha  alternative  alzheimer  alzheimer disease  american  analgesia  analgesic  analysis  analyzed  anemia  anesthesia  anesthetic  animal  anterior      anti  anti inflammatory  antibiotic  antibody  antigen  antiretroviral  anxiety  aortic  apnea  app  application  applied  \
0     NCT00058058        0.0  0.040586       0.0   0.0       0.0          0.0      0.0              0.0                          0.0            0.0  0.00000        0.0      0.0  0.052216       0.0      0.0       0.0   0.0  0.0     0.0         0.0     0.0  0.000000          0.0    0.0  0.0    0.0       0.0    0.037255           0.0      0.0             0.0       0.0        0.0  0.000000           0.0        0.000000        0.0       0.0         0.0    0.0     0.0  0.000000        0.0      0.0            0.0             0.0     0.0  0.000000      0.0  0.032396   0.0    0.0  0.0  0.000000        0.0    0.0     0.0  0.0      0.0        0.0       0.0      0.0        0.0         0.0    0.0      0.0     0.0    0.0          0.0        0.0                0.0    0.0486        0.0        0.0       0.0       0.0     0.0         0.0         0.0     0.0       0.0  0.000000                0.0         0.0       0.0      0.0             0.0      0.0     0.0    0.0  0.0          0.0      0.0   
1     NCT01980602        0.0  0.000000       0.0   0.0       0.0          0.0      0.0              0.0                          0.0            0.0  0.00000        0.0      0.0  0.000000       0.0      0.0       0.0   0.0  0.0     0.0         0.0     0.0  0.000000          0.0    0.0  0.0    0.0       0.0    0.000000           0.0      0.0             0.0       0.0        0.0  0.000000           0.0        0.000000        0.0       0.0         0.0    0.0     0.0  0.000000        0.0      0.0            0.0             0.0     0.0  0.073599      0.0  0.000000   0.0    0.0  0.0  0.000000        0.0    0.0     0.0  0.0      0.0        0.0       0.0      0.0        0.0         0.0    0.0      0.0     0.0    0.0          0.0        0.0                0.0    0.0000        0.0        0.0       0.0       0.0     0.0         0.0         0.0     0.0       0.0  0.000000                0.0         0.0       0.0      0.0             0.0      0.0     0.0    0.0  0.0          0.0      0.0   
10    NCT01277458        0.0  0.000000       0.0   0.0       0.0          0.0      0.0              0.0                          0.0            0.0  0.15232        0.0      0.0  0.000000       0.0      0.0       0.0   0.0  0.0     0.0         0.0     0.0  0.000000          0.0    0.0  0.0    0.0       0.0    0.000000           0.0      0.0             0.0       0.0        0.0  0.000000           0.0        0.000000        0.0       0.0         0.0    0.0     0.0  0.064862        0.0      0.0            0.0             0.0     0.0  0.000000      0.0  0.000000   0.0    0.0  0.0  0.000000        0.0    0.0     0.0  0.0      0.0        0.0       0.0      0.0        0.0         0.0    0.0      0.0     0.0    0.0          0.0        0.0                0.0    0.0000        0.0        0.0       0.0       0.0     0.0         0.0         0.0     0.0       0.0  0.000000                0.0         0.0       0.0      0.0             0.0      0.0     0.0    0.0  0.0          0.0      0.0   
100   NCT01164891        0.0  0.000000     

## Test merged dataframes with tf-idf results

In [ ]:
# import random

# def compare_results():
#     row = random.randint(0,3000)
#     df_tf.drop('nct_id', axis = 1)
#     print('Random row: {}'.format(row))
#     return df_tf.loc[row], tf.loc[row]

In [ ]:
# compare_results()

## Calculate correlation between docs

In [98]:
# calculate similary
similarity = tweights * tweights.T

In [99]:
# Save all data in a dataframe
df_docs_similarity = pd.DataFrame(similarity.toarray())

<b>To-do: Give a NTC-ID record, find similar documents
& return dataframe with results</b>

In [108]:
# Find more similar documents of a given record

def find_similar_docs(record, rate):
    trials_id = []
    treshold = rate
    similar_index = df_docs_similarity.iloc[record][df_docs_similarity.iloc[record] > treshold].index
    
    for i in similar_index.values:
        trials_id.append(i)
        
    print('{} similar trials with treshold {}'.format(len(trials_id), treshold))
    return df.iloc[trials_id]


In [114]:
find_similar_docs(0, 0.50)

20 similar trials with treshold 0.5


,nct_id,source,brief_title,overall_status,study_type,phase,condition,condition_browse/mesh_term,intervention_browse/mesh_term,detailed_description/textblock,brief_summary/textblock,location/facility/address/city,location/facility/address/country,location/facility/address/zip,sponsors/lead_sponsor/agency,sponsors/lead_sponsor/agency_class,study_design_info/allocation,study_design_info/intervention_model,study_design_info/primary_purpose,year_submitted,year_last_updated,year_verification,all_text,col,lemmas
0,NCT00058058,American College of Radiology Imaging Network,Magnetic Resonance Imaging in Women Recently D...,Completed,Interventional,N/A,Breast Cancer,Breast Neoplasms,None,OBJECTIVES: - Determine the d...,RATIONALE: Diagnostic procedures such as magn...,Little Rock,United States,72205,American College of Radiology Imaging Network,Other,None,Single Group Assignment,Diagnostic,2003,2018,2018.0,american college of radiology imaging network ...,https://clinicaltrials.gov/ct2/show/NCT00058058,american college of radiology imaging network ...
1452,NCT02934828,Chinese Academy of Medical Sciences,Study of Preoperative Chemotherapy for Early T...,Unknown status,Interventional,Phase 2,Breast Cancer,Breast Neoplasms,Paclitaxel,Neoadjuvant Chemotherapy (NCT) is a sta...,The investigators research aim to explores th...,Beijing,China,100021,Chinese Academy of Medical Sciences,Other,Randomized,Parallel Assignment,Treatment,2016,2016,2015.0,chinese academy of medical sciences study of p...,https://clinicaltrials.gov/ct2/show/NCT02934828,chinese academy of medical science study of pr...
1485,NCT02209389,Eventus Diagnostics Ltd,OctavaPink for Women With Dense Breast After N...,Unknown status,Observational [Patient Registry],Early Phase 1,Breast Cancer,Breast Neoplasms,None,"This is a multicenter, prospective stud...",The OctavaPink test is indicated as an adjunc...,Detroit,United States,48201,Eventus Diagnostics Ltd,Industry,None,None,None,2014,2014,2018.0,eventus diagnostics ltd octavapink for women w...,https://clinicaltrials.gov/ct2/show/NCT02209389,eventus diagnostics ltd octavapink for woman w...
1681,NCT02909361,Fudan University,Fulvestrant 500mg in Patients With Advanced Br...,Recruiting,Observational [Patient Registry],N/A,Metastatic Breast Cancer,Breast Neoplasms,Fulvestrant,"A multicenter, prospective study real-w...",Fulvestrant 500mg in Patients With Advanced B...,Shanghai,China,200032,Fudan University,Other,None,None,None,2016,2019,2015.0,fudan university fulvestrant 500mg in patients...,https://clinicaltrials.gov/ct2/show/NCT02909361,fudan university fulvestrant 500mg in patient ...
2513,NCT00002646,National Cancer Institute (NCI),Hormone Therapy in Treating Postmenopausal Wom...,Completed,Interventional,Phase 4,Breast Cancer,Breast Neoplasms,Tamoxifen,OBJECTIVES: I. Compare disease free sur...,RATIONALE: Estrogen can stimulate the growth ...,Mobile,United States,36688,Eastern Cooperative Oncology Group,Other,Randomized,None,Treatment,1999,2012,2018.0,national cancer institute (nci) hormone therap...,https://clinicaltrials.gov/ct2/show/NCT00002646,national cancer institute (nci) hormone therap...
291,NCT02136927,Sun Pharma Advanced Research Company Limited,Bioavailability Study of SPARC1210 and Referen...,Terminated,Interventional,Phase 4,Breast Cancer,Breast Neoplasms,None,None,"Pharmacokinetic, bioequivalence study",Hyderabad,India,None,Sun Pharma Advanced Research Company Limited,Industry,Randomized,Crossover Assignment,Treatment,2014,2019,2013.0,sun pharma advanced research company limited b...,https://clinicaltrials.gov/ct2/show/NCT02136927,sun pharma advanced research company limited b...
3722,NCT01895491,"Reyon Pharmaceutical Co., Ltd.",Safety Study of VM206RY in Subjects With Expre...,Completed,Interventional,Phase 2,Breast Cancer,Breast Neoplasms,None,None,The main objective of this study is to evalua...,Seoul,"Korea, Republic of",138-736,"Reyon Pharmaceutical Co., Ltd.",Industry,Non-Randomized,Single Group Assignment